In [1]:
# Импорт модулей и функций 

import re
import numpy as np
import pandas as pd
import datetime as dt

import nltk
# nltk.download('stopwords')
from nltk.corpus import stopwords
from pymorphy2 import MorphAnalyzer

import warnings
warnings.filterwarnings('ignore')

from data_preparation import find_coefs
from data_preparation import tokenizer

In [2]:
# Чтение файлов news.csv и quotes.csv
news = pd.read_csv('../_data/_raw/news.csv', sep=';')
quotes = pd.read_csv('../_data/_raw/quotes.csv', sep=';')

In [3]:
# Создание dataset из news и quotes

# В dataset попадают тикеры из news, по которым удалось собрать котировки
df_tickers = pd.DataFrame({'ticker': quotes['ticker'].unique()})
dataset = pd.merge(news, df_tickers, on='ticker', how='inner')

# И только те примеры из news, у которых title!=NaN И content!=NaN
dataset = dataset[(dataset['title'].notnull()) & (dataset['content'].notnull())]

dataset = dataset[['ticker', 'year', 'month', 'day', 'title', 'content']].reset_index()

In [12]:
# Вычисление значений к-тов a1, a2 для примеров в dataset

df_result = pd.DataFrame(columns = dataset.columns)

df = dataset.reset_index()

for T in [30]:

    for i in range(0, len(df)):
       
        row = df.iloc[i]
        dd = row['day']
        mm = row['month']
        yyyy = row['year']
        tick = row['ticker']
        
        params = {'T': T,
                  'day': dd,
                  'month': mm,
                  'year': yyyy,
                  'ticker': tick,
                  'quotes': quotes,
                  'dataset': dataset}

        T, a1, a2 = find_coefs(params)

        df.loc[i, ['T', 'a1', 'a2']] = [T, a1, a2]
        
    df_result = pd.concat([df_result, df], axis = 0)

In [ ]:
# Токенизация полей title и content

dataset = tokenizer(dataset, 'title')
dataset = tokenizer(dataset, 'content')

In [20]:
# Обновление dataset

#1 Получение news_1 - статьи, которых нет в датасете
news_1 = news.merge(dataset, how='outer', on=['year', 'month', 'day'])
news_1 = news_1[news_1['ticker_y'].isnull()][['year','month','day','ticker_x','company','title_x','content_x','url_new']]
news_1.rename(columns={'ticker_x': 'ticker', 'title_x': 'title', 'content_x': 'content'}, inplace=True)

#2 Получение dataset_1 из news_1 и quote
tickers = pd.DataFrame({ticker: quotes['ticker'].unique()})
dataset_1 = pd.merge(news_1, tickers, on='ticker', how='inner')
dataset_1 = dataset_1[(dataset_1['title'].notnull()) & (dataset_1['content'].notnull())]

#3 Проставление к-тов a1, a2
T = 30
for i, idx in enumerate(list(dataset_1.index)[:]):
    
    row = dataset_1.loc[i]
    params = {'T': T,
              'day': row['day'],
              'month': row['month'],
              'year': row['year'],
              'ticker': row['ticker'],
              'quotes': quotes,
              'dataset': dataset_1}
    
    T, a1, a2 = find_coefs(params)
    dataset_1.loc[idx, ['T', 'a1', 'a2']] = [T, a1, a2]
    print('i =', i, 'idx =', idx, 'period =', T, 'a2 =', a2)

# 4 Токенизация полей title и conten
dataset_1 = tokenizer(dataset_1, 'title')
dataset_1 = tokenizer(dataset_1, 'content')

# 5 Объединение фреймов данных dataset и dataset_1
dataset = dataset.append(dataset_1)

In [6]:
# Допроставление к-ов a2 ввиду появления свежих котировок

print('До заполнения пропусков:', 
      len(dataset[dataset['a2'].isnull()]), 
      'строк с пустым значением a2', '\n')

T = 30

for i, idx in enumerate(list(dataset.index[:])):
    row = dataset.loc[i]
    
    if np.isnan(row['a2']):
        
        params = {'T': T,
                  'day': row['day'],
                  'month': row['month'],
                  'year': row['year'],
                  'ticker': row['ticker'],
                  'quotes': quotes,
                  'dataset': dataset}
        
        T, a1, a2 = find_coefs(params)
        
        print('К-т не был рассчитан на дату:', 
               row['day'], row['month'], row['year'], 
               '\t', a2)
        
        dataset.loc[idx, ['a1', 'a2']] = a1, a2

print('После заполнения пропусков:', 
      len(dataset[dataset['a2'].isnull()]), 
      'строк с пустым значением a2', '\n')

In [ ]:
# Запись dataset в файл ../_data/dataset.v.3.2.csv
df_result.to_csv('../_data/dataset.v.3.2.csv', sep=';', index=False)